In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import datetime
import pandas as pd

In [ ]:
if(torch.cuda.is_available()):
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")
print(dev)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
training_data = []
with open('/content/drive/MyDrive/clean_data_paris_marseille.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        row[0] = float(row[0])
        if row[0]>90:
            row[0] = row[0]-360
        training_data.append([float(item) for item in row])

with open("/content/drive/MyDrive/objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)
    data = wind_data['data']


In [ ]:
data_std = 1

def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 4
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144
def int_array(arr):
    return [int(i) for i in arr]

def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_time(arr[2]),int(arr[3])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor

def input_label(training_data,dtime,i, goal_coord,data):
    array = grid_array(training_data[i])
    time = array[2]
    long = array[0]
    lat = array[1]
    alt = array[3]
    winds = data[time:time+dtime,:,:,:]
    winds = np.transpose(winds,(0,4,1,2,3))
    winds = np.arctan2(winds[:,0:1,:,:,:],winds[:,1:2,:,:,:])/(3.1415/2)
    pos = np.zeros((dtime,1,winds.shape[2],winds.shape[3],winds.shape[4]))
    for i in range(pos.shape[2]):
        pos[0,0,i,int(grid_long(goal_coord[0])),int(grid_lat(goal_coord[1]))] = -(3)
        pos[0,0,alt,long,lat] = 3
    input = np.concatenate((winds,pos),axis=1)
    action = np.arctan2(data[time,alt,long,lat,1],data[time,alt,long,lat,0])
    action = int(np.round(((action+np.pi)*360/(2*np.pi))/45)%8)


    return (input,action)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, entries, dtime, goal_coord, data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.goal_coord = goal_coord
        self.data = data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx, self.goal_coord,self.data)
        inp = np.pad(inp,((0,0),(0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

In [ ]:
coords_destination = [5.3698, 43.2965]
trainset = CustomDataset(training_data,1,coords_destination,data)
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
print(a.shape)


torch.Size([32, 1, 2, 17, 148, 73])


In [ ]:
class Net(nn.Module):
    def __init__(self , print_shape=True):
        self.print_shape = print_shape
        super(Net, self).__init__()
        self.layersCNN = nn.ModuleList( [
        nn.Conv3d(2, 64, (17,1,1)),
        nn.BatchNorm3d(64),
        nn.ReLU(),
        nn.Dropout(0.2),

        nn.Conv3d(64, 64, (1,3,3), padding=(0,1,1)),
        nn.BatchNorm3d(64),
        nn.ReLU(),
        nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2)),
        nn.Dropout(0.2),

        nn.Conv3d(64, 128, (1,3,3), padding=(0,1,1)),
        nn.BatchNorm3d(128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Conv3d(128, 128, (1,3,3), padding=(0,0,0)),
        nn.BatchNorm3d(128),
        nn.ReLU(),
        nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2),padding=(0,0,1)),

        nn.Dropout(0.2),
        nn.Conv3d(128, 256, (1,3,3), padding=(0,1,1)),
        nn.BatchNorm3d(256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Conv3d(256, 256, (1,3,3), padding=(0,0,0)),
        nn.BatchNorm3d(256),
        nn.ReLU(),
        nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2),padding=(0,0,1)),

        nn.Dropout(0.2),
        nn.Conv3d(256, 512, (1,3,3), padding=(0,0,1)),
        nn.BatchNorm3d(512),
        nn.ReLU(),
        nn.MaxPool3d(kernel_size=(1,3,3), stride=(1,2,1)),
        nn.Dropout(0.2),
        nn.Conv3d(512, 512, (1,3,3), padding=(0,0,0)),
        nn.BatchNorm3d(512),
        nn.ReLU(),
        nn.MaxPool3d(kernel_size=(1,3,3), stride=(1,2,2)),
        nn.Dropout(0.2)

        ])
        self.layers = nn.ModuleList([
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(1024, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(1024, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512, 8)])

        for l in self.layers:
            if isinstance(l, (nn.BatchNorm3d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()
            elif isinstance(l, nn.Conv3d):
                n = l.kernel_size[0] * l.kernel_size[1] * l.kernel_size[2] * l.out_channels
                l.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(l, nn.Linear):
                nn.init.xavier_normal(l.weight)

    def forward(self, x):
        x = x[:,0,:,:,:,:]
        for l in self.layersCNN:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        x = torch.flatten(x, 1)
        for l in self.layers:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        return x

In [ ]:
net = Net().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)


<ipython-input-89-99f65c6a966f>:76: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(l.weight)


In [ ]:
net = net.to(dev)

In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
r = net.to(dev)
r = net(a).to(dev)
b = b.to(dev)
z = torch.argmax(torch.softmax(r,1), axis=1).to(dev)
print(torch.softmax(r,1))
print(z)
print(b - z)
# print(r.data)
# print(b)


tensor([[0.0629, 0.1069, 0.0561, 0.0770, 0.4001, 0.0209, 0.1002, 0.1759],
        [0.1156, 0.3249, 0.0412, 0.2180, 0.1638, 0.0504, 0.0370, 0.0492],
        [0.1023, 0.0471, 0.6086, 0.0264, 0.0779, 0.0504, 0.0559, 0.0315],
        [0.2723, 0.0100, 0.1445, 0.1136, 0.1539, 0.0618, 0.2253, 0.0185],
        [0.1835, 0.3197, 0.2962, 0.0200, 0.0551, 0.0563, 0.0533, 0.0160],
        [0.0758, 0.0482, 0.3095, 0.1173, 0.0841, 0.0991, 0.1326, 0.1333],
        [0.0536, 0.0951, 0.0404, 0.1859, 0.0841, 0.0776, 0.3338, 0.1295],
        [0.0500, 0.0567, 0.2759, 0.0826, 0.2293, 0.1029, 0.0796, 0.1229],
        [0.0409, 0.0026, 0.5980, 0.0638, 0.1900, 0.0731, 0.0112, 0.0205],
        [0.0645, 0.1020, 0.1157, 0.1681, 0.0687, 0.2592, 0.1053, 0.1166],
        [0.0240, 0.2243, 0.0749, 0.0793, 0.1746, 0.0722, 0.2758, 0.0750],
        [0.0609, 0.1861, 0.2600, 0.2008, 0.1879, 0.0219, 0.0447, 0.0377],
        [0.0292, 0.0352, 0.5547, 0.0059, 0.0550, 0.0765, 0.1791, 0.0643],
        [0.1142, 0.0389, 0.0387, 0.014

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)
# exp_lr_scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=0)
    for i, data in enumerate(trainloader, 0):
      try:
        inputs, labels = data
        inputs = inputs.float().to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        if i % 10000 == 9999:    # print every 2000 mini-batches
               torch.save(net, "/content/drive/MyDrive/NET.pt")
      except Exception as e:
        print(e)



print('Finished Training')

[1,   200] loss: 0.146
[1,   400] loss: 0.140
[1,   600] loss: 0.135
[1,   800] loss: 0.129
[1,  1000] loss: 0.133
[1,  1200] loss: 0.134
[1,  1400] loss: 0.132
[1,  1600] loss: 0.126


KeyboardInterrupt: 

In [ ]:
net = torch.load("compressed2.pt")
net = net.to(dev)

In [ ]:
torch.save(net, "/content/drive/MyDrive/NET.pt")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(100):
        data = next(iter(trainloader))
        images, labels = data
        images = images.float().to(dev)
        labels = labels.to(dev)
        outputs = net(images)
        predicted = torch.argmax(torch.softmax(outputs.data, 1),axis =1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network : %d %%' % (
    100 * correct / total))

Accuracy of the network : 85 %
